# RUN MODELS ON EXAMPLE DATASET

In [1]:
import pandas as pd
import numpy as np

import zipfile
import tensorflow as tf


In [2]:
zf = zipfile.ZipFile('asl-signs.zip') 
train = pd.read_csv(zf.open('train.csv'))


,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room


In [4]:
# Add ordinally Encoded Sign (assign number to each sign name)
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [5]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [6]:
interpreter_first = tf.lite.Interpreter("./model_first.tflite")
found_signatures_first = list(interpreter_first.get_signature_list().keys())
prediction_fn_first = interpreter_first.get_signature_runner("serving_default")

In [7]:
interpreter = tf.lite.Interpreter("./model.tflite")
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")

In [8]:
interpreter_achraf = tf.lite.Interpreter("./model_achraf.tflite")
found_signatures_achraf = list(interpreter.get_signature_list().keys())
prediction_fn_achraf = interpreter.get_signature_runner("serving_default")

In [9]:
interpreter_fourmodels = tf.lite.Interpreter("./model_achraf.tflite")
found_signatures_fourmodels = list(interpreter.get_signature_list().keys())
prediction_fn_fourmodels = interpreter.get_signature_runner("serving_default")

In [12]:
pq_file = 'output.parquet'
df = pd.read_parquet('output.parquet')
xyz = pd.read_parquet(pq_file)

In [13]:
#df = remove_rows_to_make_multiple_of_543(xyz)

df.to_parquet('modified_file.pq')
xyz_np=load_relevant_data_subset('modified_file.pq')


ValueError: cannot reshape array of size 26091 into shape (16,543,3)

In [11]:
prediction_first=prediction_fn_first(inputs=xyz_np)
sign_first=prediction_first['outputs'].argmax()
probabilities_first = prediction_first['outputs']
confidence_first = probabilities_first.max()
print("PRED_first : ", ORD2SIGN.get(sign_first))# " ; confidence_first",confidence_first)

NameError: name 'xyz_np' is not defined

In [18]:
prediction=prediction_fn(inputs=xyz_np)
sign=prediction['outputs'].argmax()
probabilities = prediction['outputs']
confidence = probabilities.max()
print("PRED : ", ORD2SIGN.get(sign))# confidence",confidence)

PRED :  black


In [19]:
prediction_achraf=prediction_fn_achraf(inputs=xyz_np)
sign_achraf=prediction_achraf['outputs'].argmax()
probabilities_achraf = prediction_achraf['outputs']
confidence_achraf = probabilities_achraf.max()
print("PRED_achraf : ", ORD2SIGN.get(sign))#, " ; confidence_achraf",confidence_achraf)

PRED_achraf :  black


In [20]:
prediction_fourmodels=prediction_fn_fourmodels(inputs=xyz_np)
sign_fourmodels=prediction_fourmodels['outputs'].argmax()
probabilities_fourmodels = prediction_fourmodels['outputs']
confidence_fourmodels = probabilities_fourmodels.max()
print("PRED_fourmodels : ", ORD2SIGN.get(sign))#, " ; confidence_fourmodels",confidence_fourmodels)


PRED_fourmodels :  black
